# Training the competition model

In [5]:
original_json_file_path = "./datasets/competition_final/generated/final_year_1_to_10_data.json"
year_11_json_file_path = "./datasets/competition_final/generated/final_year_11_data.json"

In [7]:
import pandas as pd

df_original = pd.read_json(original_json_file_path)
df_year_11 = pd.read_json(year_11_json_file_path)

df_year_11.head()

,tmID,year,playoff,averageWinRate,averagePoints,averageRebounds,averageAssists,averageSteals,averageBlocks,averageTurnovers,averageFGRatio,averageFTRatio,averageThreeRatio,coachWinRate,numberOfAwardedPlayers
0,WAS,11,NaN,0.470588,244.629371,122.927739,47.384615,29.587413,12.310023,50.428904,0.418658,0.749509,0.280573,0.500000,2
1,TUL,11,NaN,0.500000,196.262500,87.358333,39.875000,19.675000,8.566667,38.229167,0.414552,0.780216,0.327587,0.500000,10
2,SEA,11,NaN,0.588235,274.577622,106.679720,55.923077,26.044755,11.078322,49.041958,0.412312,0.802031,0.310246,0.588235,8
3,SAS,11,NaN,0.441176,318.134545,111.044848,67.100000,34.910909,14.202424,50.506061,0.428546,0.808280,0.340362,0.441176,2
4,PHO,11,NaN,0.676471,259.051748,113.709557,49.538462,21.234965,13.693240,47.233100,0.444417,0.801936,0.335072,0.676471,6
